In [1]:
import numpy as np
import math

from numpy import pi
from sympy import *

---

In [2]:
class IntegralForDirichlet:
    
    def __init__(self, func, curve, domain, iterations, precision = 0.01):
        self.func, self.funcSymbols = func
        self.curve = np.array(curve)
        self.domain = domain
        self.N, self.n = iterations
        
        self.__initGradient()
        self.__initCurveDerivatives()
        
# Initializaton functions
#----------------------------------------------------------------------        
    
    def __initCurveDerivatives(self):
        self.curve1D = self.curveDiff(self.curve)
        self.curve2D = self.curveDiff(self.curve1D)
        
    def __initGradient(self):
        self.funcGradient = self.gradientDiff(self.func)
        


# Main functions
#----------------------------------------------------------------------        
 
    def solve(self):
        core = 0
        
        aK = 1 / self.N
        
        for k in range(1, self.N):
            etaK = 1 - (2*k-1)/(2*self.N)
            
            for j in range(2 * self.n - 1):
                tK = j*pi / self.n
                
                core += aK * self.getCore(self.parametrize(0.3, pi/3), self.parametrize(etaK, tK))
        
        return 1/(2*self.n) * abs(core)
    
    
    def getCore(self, x,y):
        eta, _ = x
        ksi, _ = y
        
        return self.kernel(x,y) if eta != ksi else self.kernelSingular(x,y)
        
    
    def kernel(self, x,y): return np.dot( (x-y), self.gradient(x) ) * self.jacobian(y) \
                                    / ( self.functionSubs(self.func, y) * (np.linalg.norm(x-y)**2) )
    
        
    def kernelSingular(self, x,y):
        eta, t = x
        ksi, tau = y
        
        self.calculateSingularFunctions(t, tau)
                      
        firstPart = np.dot( self.gradient(eta*self.curveT), self.normalVector ) \
                        * self.innerKernelOne(t, tau)
        
        secondPart = (1 / self.euclideanDistance) \
                        * np.dot( self.gradient(eta*self.curveT), self.tangentVector ) \
                        * self.innerKernelTwo(t, tau)
        
        thirdPart = (1 / (2*self.euclideanDistance)) \
                        * np.dot( self.gradient(eta*self.curveT), self.tangentVector )
        
        denominator = eta*self.functionSubs(self.func, eta*self.curveTau)
        
        jacobian = self.jacobian(y)
        cot = self.singularCot(t, tau)
        
        return (firstPart - secondPart - thirdPart) * cot * jacobian  / denominator 
    
#----------------------------------------------------------------------      

    def innerKernelOne(self, t, tau):
        
        if t == tau:
            vector = np.flip(self.curve1DT)
            
            return np.multiply( self.negateAt(vector, 0), self.curve2DT ) / ( 2 * self.euclideanDistance ) ** 3
                        
        return np.dot( self.curveT - self.curveTau, self.normalVector ) / np.linalg.norm( self.curveT - self.curveTau ) ** 2
        
        
    def innerKernelTwo(self, t, tau):
        
        if t == tau:
            return np.dot(self.curve1DT, self.curve2DT) / ( 2 * self.euclideanDistance ) ** 2     
        
        return np.dot( self.curveT - self.curveTau, self.curve1DT ) / np.linalg.norm( self.curveT - self.curveTau ) ** 2  \
            - sin(tau -t) / ( 2 * (1 - cos(tau-t) ))
    
    
    
    def calculateSingularFunctions(self, t, tau):
        self.normalVector = self.normal(t)
        self.tangentVector = self.tangent(t)
        
        self.euclideanDistance = np.linalg.norm(self.curveSubs(self.curve1D, t))
        
        self.curveT = self.curveSubs(self.curve, t)        
        self.curveTau = self.curveSubs(self.curve, tau)
        
        self.curve1DT = self.curveSubs(self.curve1D, t)
        self.curve2DT = self.curveSubs(self.curve2D, t)
        
        
  
    
#----------------------------------------------------------------------         

    def parametrize(self, eta, t): return eta * self.curveSubs(self.curve, t)
        

    def normal(self, value):
        
        vector = np.flip(self.curveSubs(self.curve1D, value))
        
        vector = self.negateAt(vector, -1)
        
        return vector / np.linalg.norm(vector)
    
    
    def tangent(self, value):
        
        vector = self.curveSubs(self.curve1D, value)
        
        return vector / np.linalg.norm(vector)
    
    
    def gradient(self, vector): 
        return np.array([ self.functionSubs(function, vector) for function in self.funcGradient ])
    
    
    def jacobian(self, vector):
        ksi, tau = vector
        
        innerVector = np.multiply( self.curveSubs(self.curve, tau), np.flip( self.curveSubs(self.curve1D, tau)) )
        
        return ksi * (innerVector[0] - innerVector[-1])
    
    
    def singularCot(self, t, tau): 
        return - 1/self.n * sum([ sin(m*(t-tau)) for m in range(self.n) ]) - 1/(2*self.n) * sin(n*(t-tau))


# Helper functions
#----------------------------------------------------------------------     

    def functionSubs(self, function, vector): return float(function.subs(zip(self.funcSymbols, vector)))

    def curveSubs(self, curve, value): return np.array([ float(function.subs({s:value})) for function in curve ])
           
    def curveDiff(self, curve): return np.array([ function.diff() for function in curve ])
    
    def gradientDiff(self, function): return np.array([ function.diff(symbol) for symbol in self.funcSymbols ])    
    
    def negateAt(self, vector, index): 
        
        vector[index] = vector[index] * -1
        
        return vector
    

---

## Initialization

In [3]:
s = Symbol("s")                           # substitution value
funcSymbols = [a, b] = symbols("a, b")    # func substitution values

---

In [4]:
func = (a+b+2)**2, funcSymbols
curve = x1, x2 = cos(s), sin(s)
domain = (0, 1), (0, 2*pi)
iterations = N, n = 10, 256

## Calculation

In [5]:
integralSolver = IntegralForDirichlet(func, curve, domain, iterations)


print('\n==============================\n')

result = integralSolver.solve()
print(f'Result = {float(result)}')

print('\n==============================\n')



Result = 0.6946521147089756


